# Différentes approches de NLP

## Données

In [1]:
import pandas as pd
df = pd.read_csv("../../../data/tableau_presse.csv")

## Regex

Extraire toutes les dates 

In [15]:
import re
m = '(\d{1,2}\s+\w+\s+\d{4})'

In [16]:
texte = df["Contenu_mod"].iloc[0]

In [17]:
re.findall(m, texte)

['11\xa0février\xa02025', '24 et 2027']

## Mot par mot, ou dictionnaires

comptons certains mots 
- startup
- entreprise
- etc.

In [19]:
mots = ["intelligence artificielle", "startup", "informatique"]

## DFM

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

In [23]:
modele = CountVectorizer()

modele.fit(df["Contenu_mod"].dropna())

CountVectorizer()

In [24]:
modele.get_feature_names_out()

array(['00', '000', '000e', ..., 'œuvrecette', 'œuvrer', 'œuvres'],
      dtype=object)

In [26]:
tableau = modele.transform(df["Contenu_mod"].dropna())

In [30]:
pd.DataFrame(tableau.toarray(), columns=modele.get_feature_names_out())

,00,000,000e,001,002,003,004,00bienvenueen,00la,01,...,ôtant,ôte,ôtent,œil,œuf,œufs,œuvre,œuvrecette,œuvrer,œuvres
0,0,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,14,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
991,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
992,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
993,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Pas mal d'options

In [31]:
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop

In [33]:
#fr_stop

## TF-IDF

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn(


Peut être pas la meilleure échelle : décomposer au paragraphe ?

## Premiers pas avec SpaCy


### Télécharger et utiliser un modèle (un trained pipepeline)

Se familiariser avec la structure doc/span/token

In [35]:
#pip install spacy

Installer le pipeline

In [37]:
#!python -m spacy download fr_core_news_lg

In [39]:
import spacy
pipeline = spacy.load("fr_core_news_lg")

In [41]:
pipeline

In [42]:
doc1 = pipeline("J'aime beaucoup programmer en Python")

In [48]:
for m in doc1:
    print(m, m.lemma_, m.pos_)

J' je PRON
aime aimer VERB
beaucoup beaucoup ADV
programmer programmer VERB
en en ADP
Python Python PROPN


In [49]:
for m in doc1:
    if m.pos_ == "VERB":
        print(m)

aime
programmer


Possibilité d'avoir la distance entre deux textes

In [50]:
# Compare two documents
doc1 = pipeline("J'aime beaucoup programmer en Python")
doc2 = pipeline("J'aime beaucoup programmer en R")
print(doc1.similarity(doc2))

0.9162800610031004


In [53]:
txt_emb = pipeline(texte)

In [55]:
#txt_emb.ents

### Lemmatiser et garder que les noms

Utiliser le POS

In [8]:
# Partie à faire en TP

### Identifier les entités nommées

In [ ]:
text = df["text"].iloc[0]
doc = pipeline(text)
for ent in doc.ents:
    print(ent.text, ent.label_)

Afficher les entités nommées.

In [ ]:
from spacy import displacy
displacy.render(doc, style="ent", 
                jupyter=True)

## GliNER

Un modèle plus complexe pour l'identification d'entités

In [57]:
#!pip install gliner

In [59]:
from gliner import GLiNER
model = GLiNER.from_pretrained("urchade/gliner_multi")

/opt/anaconda3/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [67]:
# Labels for entity prediction
labels = ["politiciens","companies","organizations"]

# Perform entity prediction
entities = model.predict_entities(texte, labels, threshold=0.5)

entities

/opt/anaconda3/lib/python3.11/site-packages/gliner/modules/base.py:33: UserWarning: Sentence of length 1089 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")


[{'start': 69,
  'end': 84,
  'text': 'Emmanuel Macron',
  'label': 'politiciens',
  'score': 0.9890307784080505},
 {'start': 335,
  'end': 341,
  'text': 'Elysée',
  'label': 'organizations',
  'score': 0.5381392240524292},
 {'start': 917,
  'end': 923,
  'text': 'Elysée',
  'label': 'organizations',
  'score': 0.6087769865989685}]